<a href="https://colab.research.google.com/github/nishitjain97/Example-Based-Neural-Machine-Translation-System/blob/master/4_Seq2Seq_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive onto Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, CuDNNLSTM
from keras.models import Model
import tensorflow as tf
import pickle

Using TensorFlow backend.


In [0]:
with open('/content/drive/My Drive/English Dataset/IITB.en-hi_cleaned.en') as f:
  sentences = f.readlines()
  
lines = pd.DataFrame(sentences, columns=['Src'])

lines['Tar'] = None

with open('/content/drive/My Drive/English Dataset/IITB.en-hi_cleaned.hi') as f:
  lines['Tar'] = f.readlines()

In [0]:
lines.Src = lines.Src.apply(lambda x: x.strip())
lines.Tar = lines.Tar.apply(lambda x: x.strip())
lines.Tar = '<START> ' + lines.Tar + ' <END>'

In [6]:
lines.shape

(10000, 2)

In [7]:
lines.sample(10)

,Src,Tar
7758,generic gcj compiled java,<START> generic gcj compiled java <END>
2058,move a onto the nine of clubs,<START> a को चिड़ी का नहला पर ले जाएँ <END>
1491,maximum value,<START> अधिकतम मान <END>
5899,aim to place the suits in the order which fits...,<START> वर्तमान विन्यास अति प्राकृतिक रूप से त...
3270,red joker,<START> लाल जोकर <END>
8559,conflicted,<START> विरोधी <END>
3486,the king of diamonds,<START> ईंट का बादशाह <END>
6371,documents,<START> खाली दस्तावेज <END>
7137,default project directory,<START> कार्य डायरेक्टरी <END>
4090,the game file to use,<START> खेल फ़ाइल उपयोग में <END>


In [0]:
with open('/content/drive/My Drive/English Dataset/embeddings.en', 'rb') as f:
    src_summary = pickle.load(f)
    
with open('/content/drive/My Drive/English Dataset/embeddings.hi', 'rb') as f:
    tar_summary = pickle.load(f)

In [0]:
length_of_dict = len(tar_summary['dictionary'])
tar_summary['dictionary']['<START>'] = length_of_dict
tar_summary['reverse_dictionary'][length_of_dict] = '<START>'
tar_summary['embeddings'] = np.vstack((tar_summary['embeddings'], np.zeros((tar_summary['embeddings'].shape[1]))))

length_of_dict = len(tar_summary['dictionary'])
tar_summary['dictionary']['<END>'] = length_of_dict
tar_summary['reverse_dictionary'][length_of_dict] = '<END>'
tar_summary['embeddings'] = np.vstack((tar_summary['embeddings'], np.zeros((tar_summary['embeddings'].shape[1]))))

In [0]:
all_src_words = src_summary['dictionary'].keys()
all_tar_words = tar_summary['dictionary'].keys()

In [11]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Src])
max_len_src

41

In [12]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Tar])
max_len_tar

42

In [13]:
num_encoder_tokens = len(all_src_words)
num_decoder_tokens = len(all_tar_words)
(num_encoder_tokens, num_decoder_tokens)

(100000, 500002)

In [14]:
lines = shuffle(lines)
lines.head(10)

,Src,Tar
8136,location,<START> स्थान <END>
2289,move a card or build of cards on to the empty ...,<START> एक पत्ता या पत्तों के समूह खाली खाँचा ...
3740,d,<START> d not applicablenot applicable <END>
736,bottom panel,<START> निचला पटल <END>
7020,add,<START> file roller gnome hi po file roller hi...
4480,move a onto the ace of diamonds,<START> ईंट का इक्का <END>
1348,key,<START> कुंजी <END>
1775,hopscotch,<START> हॉपस्कॉच <END>
9758,symboldb menu actions,<START> नमूना क्रिया <END>
6649,new file,<START> नया फिल्टर <END>


In [15]:
X, y = lines.Src, lines.Tar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape, X_test.shape

((9000,), (1000,))

In [0]:
del lines

In [0]:
def generate_batch(X=X_train, y=y_train, batch_size=128):
  while True:
    for j in range(0, len(X), batch_size):
      encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
      decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
      decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
      
      for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
        for t, word in enumerate(input_text.split()):
          if word not in src_summary['dictionary'].keys():
            word = 'UNK'
          encoder_input_data[i, t] = src_summary['dictionary'][word]
        
        for t, word in enumerate(target_text.split()):
          if word not in tar_summary['dictionary'].keys():
            word = 'UNK'
          
          if t < len(target_text.split())-1:
            decoder_input_data
            
          if t > 0:
            decoder_target_data[i, t-1, tar_summary['dictionary'][word]] = 1
            
        yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
latent_dim = 128

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

def training():
  """
    Defines a Keras model and runs on training set give by generator function.
  """
  ### Encoder ###
  # Input layer
  encoder_inputs = Input(shape=(None, ), name='Encoder_Inputs')
  
  # Word2Vec embedding layer
  encoder_embedding_layer = Embedding(num_encoder_tokens, latent_dim, 
                                      weights=[src_summary['embeddings']], 
                                      name='English_Embeddings')
  encoder_embeddings = encoder_embedding_layer(encoder_inputs)
  
  # Encoder LSTM layer
  encoder_lstm = CuDNNLSTM(64, return_state=True, name='Encoder_LSTM')
  encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)
  
  # Save states for decoder
  encoder_states = [state_h, state_c]
  
  
  ### Decoder ###
  # Input layer
  decoder_inputs = Input(shape=(None, ), name='Decoder_Inputs')
  
  # Word2Vec embedding layer
  decoder_embedding_layer = Embedding(num_decoder_tokens, latent_dim, 
                                      weights=[tar_summary['embeddings']], 
                                      name='Hindi_Embeddings')
  decoder_embeddings = decoder_embedding_layer(decoder_inputs)
  
  # Decoder LSTM layer
  decoder_lstm = CuDNNLSTM(64, return_sequences=True, return_state=True, name='Decoder_LSTM')
  decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)
  
  # Fully connected Softmax layer for predictions
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense')
  decoder_outputs = decoder_dense(decoder_outputs)
  
  
  ### Model ###
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
  model.fit_generator(generator=generate_batch(X_train, y_train, batch_size=batch_size), 
                      steps_per_epoch=train_samples//batch_size, 
                      epochs=epochs, 
                      validation_data=generate_batch(X_test, y_test, batch_size=batch_size), 
                      validation_steps=val_samples//batch_size)
training()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10


In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hin_summary['dictionary']['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = hin_summary['reverse_dictionary'][sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [0]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][7:-5])
print('Predicted Hindi Translation:', decoded_sentence[:-5])

Input English sentence: too many selectable children
Actual Hindi Translation:  बहुत अधिक चयनीय शिशु हैं 
Predicted Hindi Translation:  बहुत अधिक चयनीय शिशु हैं 


In [1]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: c4dbbf68-7cc1-4958-81c9-84080122875b
Adapting to protocol v5.1 for kernel c4dbbf68-7cc1-4958-81c9-84080122875b
Adapting to protocol v5.1 for kernel c4dbbf68-7cc1-4958-81c9-84080122875b
2019-04-01 14:36:25.486024: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-01 14:36:25.486495: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x19c2520 executing computations on platform Host. Devices:
2019-04-01 14:36:25.486530: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-01 14:36:25.633812: I tensorflow/stream_exec